In [1]:
import pymysql
from sqlalchemy import create_engine
import requests
from bs4 import BeautifulSoup
import lxml
import datetime

In [2]:
class Data_gathering:
    def __init__(self):
        #importing libraries, packages and functions
        import pymysql
        from sqlalchemy import create_engine
        import requests
        from bs4 import BeautifulSoup
        import lxml
        import datetime
        #defining url-s with data sources
        self.url_lotto = 'http://megalotto.pl/najwyzsze-wygrane/lotto'
        self.url_polish_cities = 'https://www.polskawliczbach.pl/Miasta'
        self.url_polish_provinces = 'https://www.polskawliczbach.pl/Wojewodztwa'
        
    def sql_connection(self):
        self.user = str(input('Enter user name: '))
        self.password = str(input('Enter password: '))
        self.database = str(input('Enter database name: '))
        self.conn = pymysql.connect("localhost", self.user, self.password, self.database)
        self.c = self.conn.cursor()
        print('Connected to SQL', self.database, 'database')
        
        

In [3]:
lotto = Data_gathering()

In [4]:
lotto.sql_connection()

Enter user name:  Piotrek
Enter password:  alamakota
Enter database name:  project_lotto


Connected to SQL project_lotto database
